In [1]:
from random import choices, randint, randrange, random, sample 
from collections import namedtuple
from typing import List, Optional, Callable, Tuple
#Choices(, , # of draws from the population (2 , a pair)) is a function from the random module of python

In [2]:
###########################################################################
#========================> 0.Chromosome intialisation #==============================>
##########################################################################
#1. Initilize Chromosome, ie, candidate solutions placeholder list and specify type
ChromoSpeed = List[int] #list of 1 and 0 ?? ; need to update !!
ChromoVessel = List[int]

#2.Initiliase Population, ie multiple candidate solution placeholder list with Chromosome type   
PopulationSpeed = List[ChromoSpeed] # list of candidate solutions generation=pop={solu1, ..., soln}={Chromosome1, ..., Chromosome} 
PopulationVessel = List[ChromoVessel]


In [3]:
###########################################################################
#========================> 1.Callable functions #==============================>
###########################################################################
#callable fn Use functions as parameters for abstracting the problem from the algo
#===============================================================================#
#===============================================================================#
PopulateFunc = Callable[[], Tuple[PopulationSpeed, PopulationVessel]] # a population fn that takes nothing and returns new solutiobns #=Callable[[Input], Output]
#===============================================================================#
#===============================================================================#
FitnessFunc = Callable[[ChromoSpeed, ChromoVessel], int] # a fitness function that takes a Chromosome and returns a fitness value to make the correct choice
#===============================================================================#
SelectionFunc = Callable[[PopulationSpeed, PopulationVessel, FitnessFunc], Tuple[ChromoSpeed, ChromoSpeed, ChromoVessel,ChromoVessel]]  #takes a population and a fitness fn to select 2 solutions to be the parents of our next generaation solution
#===============================================================================#
CrossoverFunc = Callable[[ChromoSpeed, ChromoSpeed, ChromoVessel, ChromoVessel], Tuple[ChromoSpeed, ChromoSpeed, ChromoVessel, ChromoVessel]] #takes 2 Chromosome and returns 2 new Chromosome 
#===============================================================================#
MutationFunc = Callable[[ChromoSpeed, ChromoVessel], Tuple[ChromoSpeed, ChromoVessel]] #takes 1 Chromosome and sometimes returns a modified one
#===============================================================================#
PrinterFunc = Callable[[PopulationSpeed, PopulationVessel, int, FitnessFunc], None]


In [5]:
###########################################################################
#========================> 1.Encoding and Generating Chromosome #==============================>
###########################################################################
#######How to encode vessel speed and number of vessels #############
#Implement value encoding for vessel speed 
#Implement binary encoding for number of vessels 

def generate_chromo_speed(length: int) -> ChromoSpeed:
    '''A function to encode and generate chromosomes for one solution of vessel speed k is length of the chromosome'''
    
    return choices([0,1], k=length)

def generate_chromo_vessel(length: int) -> ChromoVessel:
    '''A function to encode and generate chromosome for one solution of number of vessels
        k is length of the chromosome'''
    
    return choices([0,1], k=length)